## 39. 数组中出现次数超过一半的数字

数组中有一个数字出现的次数超过数组长度的一半，请找出这个数字。例如输入一个长度为9的数组[1,2,3,2,2,2,5,4,2]。由于数字2在数组中出现了5次，超过数组长度的一半，因此输出2。如果不存在则输出0。

### 分析
这道题的第一想法可能是先将数组排序，这样就能很容易地统计出每个数字出现的次数。而排序的时间复杂度是$O(n\log n)$。接下来我们尝试找出更快的算法。

[//]: # (<img src="images/img123.png" style="width: 400px;"/>)

### 解法一：基于Partition函数的时间复杂度为$O(n)$的算法

如果考虑数组的特性：数组中有一个数字出现的次数超过了数组长度的，也就是说如果对数组排序那么出现次数最多的数字一定是位于数组中间。

受[Quick Sort](https://www.geeksforgeeks.org/quick-sort/)的启发，在随机快速排序算法中，我们先在数组中随机选择一个数字，然后调整整数组中数字的顺序，使得比选中的数字小的数字都排在它的左边，比选中的数字大的数字都排在它的右边。如果选中的数字的下表刚好是`n/2`，那么这个数字就是中位数；如果它的下标大于`n/2`，那么中位数应该位于它的左边，我们可以接着在它的左边部分查找。如果下标小于`n/2`，同理。

In [9]:
def more_than_half_num(numbers):
    if check_invalid_array(numbers):
        return 0
    
    middle = len(numbers) >> 1 #half of the length
    start = 0
    end = len(numbers) - 1
    index = partition(numbers, start, end)
    
    while index != middle:
        if index > middle:
            end = index - 1
            index = partition(numbers, start, end)
        else:
            start = index + 1
            index = partition(numbers, start, end)
    
    result = numbers[middle]
    if not check_more_than_half(numbers, result):
        result = 0
    return result

def check_invalid_array(numbers):
    input_invalid = False
    if numbers is None or len(numbers) == 0:
        input_invalid = True
    return input_invalid

def check_more_than_half(numbers, num):
    times = 0
    for i in range(len(numbers)):
        if numbers[i] == num:
            times += 1
    
    is_more_than_half = True
    if times * 2 <= len(numbers):
        is_more_than_half = False
    
    return is_more_than_half

################################
# partition algorithm          #
################################
def partition(data, start, end):
    """
    This function takes last element as pivot, places
    the pivot element at its correct position in sorted
    array, and places all smaller (smaller than pivot)
    to left of pivot and all greater elements to right
    of pivot
    """
    if data is None or len(data) <= 0 or start < 0 or end >= len(data):
        raise Exception('Invalid Parameters')
    
    # pivot (Element to be placed at right position)
    pivot = data[end]
    i = start - 1 # index of smaller element
    
    for j in range(start, end):
        # if current element is smaller than or equal to pivot
        if data[j] <= pivot:
            i += 1 # increase the index of smaller element
            data[i], data[j] = data[j], data[i]
            
    data[i+1], data[end] = data[end], data[i+1]
    return i + 1
    

In [10]:
# Test
test1 = [1, 2, 3, 2, 2, 2, 5, 4, 2]
print(more_than_half_num(test1))

2


###  解法二： 根据数组特点找出时间复杂度为O(n)的算法
数组中有一个数字出现的次数超过数组长度的一半，也就是说它出现的次数比**其他所有数字**出现次数的和还要多。因此我们可以考虑在遍历数组的时候保存两个值：一个是数组中的一个数字，另一个是次数。

当我们遍历到下一个数字的时候，
- 如果下一个数字和我们前一个保存的数字相同，则次数加1
- 如果下一个数字和我们前一个保存的数字不同，则次数减1
- 如果次数为零，那么我们需要保存下一个数字，并把次数设为1

由于我们要找的数字出现的次数比其他所有数字出现的次数之和还要多，那么要找的数字肯定是最后一次把次数设为1时对应的数字

In [16]:
def more_than_half_num2(numbers):
    if check_invalid_array(numbers):
        return 0
    
    result = numbers[0]
    times = 1
    
    for i in range(1, len(numbers)):
        if times == 0:
            result = numbers[i]
            times = 1
        elif numbers[i] == result:
            times += 1
        else:
            times -= 1
    
    if not check_more_than_half(numbers, result):
        result = 0
    
    return result

### Test

In [17]:
# Test
test1 = [1, 2, 3, 2, 2, 2, 5, 4, 2]
print(more_than_half_num2(test1))

2


### 解法三：自己瞎整的

上面的两个方法都是找到一个出现次数最多的数字（这里多多少少已经要遍历一遍数组），然后用`check_more_than_half()`函数**把整个数组遍历一遍**以确定这个数组是不是出现了一半长度以上。

所以为什么不直接先遍历一遍数组，把每个数字的次数都记下来（用dictionary），然后这样不是更好比较？

In [21]:
def more_than_half_num3(numbers):
    if check_invalid_array(numbers):
        return 0
    
    elem_count_dict = {}
    for i in range(len(numbers)):
        if numbers[i] in elem_count_dict.keys():
            elem_count_dict[numbers[i]] += 1
        else:
            elem_count_dict[numbers[i]] = 1
    
    # find the max_count in the dictionary and check if the max_count >= half of the len(numbers)
    max_count = max(elem_count_dict.values())
    if max_count > len(numbers) // 2:
        for elem, count in elem_count_dict.items():
            if count == max_count:
                return elem
    else:
        return 0
    

In [22]:
# Test
test1 = [1, 2, 3, 2, 2, 2, 5, 4, 2]
print(more_than_half_num3(test1))

2
